# LA County coronavirus vaccine administration by city/community

### Load Python tools

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import geopandas as gpd
import altair as alt
import altair_latimes as lat
import glob
import path
import os

In [3]:
alt.themes.register("latimes", lat.theme)
alt.themes.enable("latimes")
pd.options.display.max_columns = 50
pd.options.display.max_rows = 1000
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

### Get the county-level data from LA County

In [4]:
# Pulled from here: http://www.publichealth.lacounty.gov/media/coronavirus/locations.htm

In [5]:
url = "http://www.publichealth.lacounty.gov/media/coronavirus/locations.htm#vaccinated"

In [6]:
tables = pd.read_html(url)

In [7]:
df = tables[5]

In [8]:
df.columns = (
    df.columns.str.strip()
    .str.lower()
    .str.replace(" ", "_")
    .str.replace("(", "")
    .str.replace("*", "")
    .str.replace("-", "_")
)

---

### Create a timeseries from coronavirus-tracker data

In [9]:
path = "/Users/mhustiles/data/github/coronavirus-tracker/_notebooks/data/raw/vaccines/los-angeles/"
all_files = glob.glob(os.path.join(path, "*.csv"))

In [10]:
df_from_each_file = (
    pd.read_csv(f, encoding="ISO-8859-1", low_memory=False) for f in all_files
)
concatenated_df = pd.concat(df_from_each_file, ignore_index=True)

In [11]:
df_time = concatenated_df[
    (concatenated_df["vaccinations"] != "Unreliable Data")
    & (concatenated_df["vaccinations"] != "<5")
].copy()

In [12]:
df_time.head()

,area,vaccinations,pop_2019,vaccinations_pct,update_date
0,City of Agoura Hills,5008,17447,28.7,2021-03-25
1,City of Alhambra,18947,71313,26.6,2021-03-25
2,City of Arcadia,13777,49666,27.7,2021-03-25
3,City of Artesia,3571,13877,25.7,2021-03-25
4,City of Avalon,317,3000,10.6,2021-03-25


In [13]:
# df_time["vaccinations_pct"] = (
#     df_time["vaccinations_pct"]
#     .fillna("0")
#     .str.replace(".", "", regex=False)
#     .str.replace("No Pop Data", "", regex=False)
#     .astype(float)
# )

In [14]:
# df_time["pop_2019"] = (
#     df_time["pop_2019"]
#     .str.replace("No Pop Data", "", regex=False)
#     .fillna("0")
#     .astype(float)
# )

In [15]:
df_time["vaccinations"] = df_time["vaccinations"].astype(float)

In [16]:
df_time["pop_2019"] = df_time["pop_2019"].astype(float)

In [17]:
df_time["vaccinations_pct"] = (
    (df_time["vaccinations"] / df_time["pop_2019"]) * 100
).round(2)

In [18]:
df_time["week"] = pd.to_datetime(df_time["update_date"]).dt.week

In [19]:
# df_time["week"] = df_time["week"].astype(str)

In [20]:
df_time.week.value_counts()

13    2352
12    2352
11    2352
10    2342
9     2338
14    1677
8     1670
Name: week, dtype: int64

In [21]:
df_time.sort_values("week", ascending=False)

,area,vaccinations,pop_2019,vaccinations_pct,update_date,week
3475,Unincorporated - White Fence Farms,743.0,3272.0,22.71,2021-04-07,14
3479,Unincorporated - Wiseburn,1590.0,4721.0,33.68,2021-04-07,14
1400,City of Bell Gardens,6554.0,30840.0,21.25,2021-04-05,14
1399,City of Bell,5994.0,26746.0,22.41,2021-04-05,14
1398,City of Baldwin Park,17077.0,58253.0,29.32,2021-04-05,14
...,...,...,...,...,...,...
8467,Los Angeles - Eagle Rock,5570.0,NaN,NaN,2021-02-24,8
8468,Los Angeles - East Hollywood,2426.0,NaN,NaN,2021-02-24,8
8469,Los Angeles - Echo Park,1259.0,NaN,NaN,2021-02-24,8
8470,Los Angeles - El Sereno,4292.0,NaN,NaN,2021-02-24,8


In [22]:
df_time_pivot = (
    pd.pivot_table(
        df_time,
        values="vaccinations_pct",
        index=["area", "pop_2019"],
        columns="week",
        aggfunc="sum",
    )
    .round(2)
    .reset_index()
)

In [23]:
df_time_pivot.head()

week,area,pop_2019,8,9,10,11,12,13,14
0,City of Agoura Hills,17447.0,65.46,152.74,166.50,200.90,231.53,272.37,155.64
1,City of Agoura Hills,18019.0,NaN,NaN,NaN,NaN,NaN,NaN,47.18
2,City of Alhambra,71313.0,56.04,130.76,146.54,185.99,218.33,261.45,149.40
3,City of Alhambra,72940.0,NaN,NaN,NaN,NaN,NaN,NaN,45.10
4,City of Arcadia,49666.0,62.70,146.30,159.98,194.18,223.94,263.62,150.64


In [24]:
df_time_pivot.columns = ["week_" + str(col) for col in df_time_pivot.columns]

In [25]:
df_time_pivot.rename(
    columns={"week_area": "area", "week_pop_2019": "population"}, inplace=True
)

In [26]:
df_time_pivot.head()

,area,population,week_8,week_9,week_10,week_11,week_12,week_13,week_14
0,City of Agoura Hills,17447.0,65.46,152.74,166.50,200.90,231.53,272.37,155.64
1,City of Agoura Hills,18019.0,NaN,NaN,NaN,NaN,NaN,NaN,47.18
2,City of Alhambra,71313.0,56.04,130.76,146.54,185.99,218.33,261.45,149.40
3,City of Alhambra,72940.0,NaN,NaN,NaN,NaN,NaN,NaN,45.10
4,City of Arcadia,49666.0,62.70,146.30,159.98,194.18,223.94,263.62,150.64


In [27]:
df_time_pivot["change_week8_to_14"] = (
    ((df_time_pivot["week_14"] - df_time_pivot["week_8"]) / df_time_pivot["week_8"])
    * 100
).round()

In [28]:
df_time_pivot.sort_values("change_week8_to_14", ascending=False).head(10)

,area,population,week_8,week_9,week_10,week_11,week_12,week_13,week_14,change_week8_to_14
574,Unincorporated - Palmdale,589.0,6.63,15.47,18.85,27.30,45.66,70.14,40.08,505.0
456,Unincorporated - Angeles National Forest,1075.0,18.15,42.35,46.43,56.63,96.26,149.10,85.20,369.0
161,City of Vernon,222.0,75.69,176.61,201.83,264.88,405.40,592.76,338.72,348.0
398,Los Angeles - University Park,23588.0,19.38,45.22,53.54,74.34,102.60,140.28,80.16,314.0
516,Unincorporated - Florence-Firestone,45509.0,20.28,47.32,55.54,76.09,103.51,140.07,80.04,295.0
524,Unincorporated - Hi Vista,852.0,15.84,36.96,40.48,49.28,74.63,108.43,61.96,291.0
617,Unincorporated - South El Monte,1340.0,24.39,56.91,69.01,99.26,128.15,166.67,95.24,290.0
554,Unincorporated - Littlerock/Pearblossom,2757.0,16.77,39.13,45.57,61.67,83.99,113.75,65.00,288.0
546,Unincorporated - Lennox,16313.0,21.87,51.03,60.57,84.42,111.57,147.77,84.44,286.0
615,Unincorporated - South Antelope Valley,413.0,17.43,40.67,48.91,69.51,89.85,116.97,66.84,283.0


In [29]:
df_time_pivot.sort_values("change_week8_to_14", ascending=False).tail(20)

,area,population,week_8,week_9,week_10,week_11,week_12,week_13,week_14,change_week8_to_14
632,Unincorporated - Twin Lakes/Oat Mountain,1394.0,NaN,NaN,NaN,NaN,NaN,NaN,39.81,NaN
633,Unincorporated - Universal City,0.0,NaN,NaN,NaN,NaN,inf,inf,inf,NaN
635,Unincorporated - Val Verde,2720.0,NaN,NaN,NaN,NaN,NaN,NaN,26.47,NaN
637,Unincorporated - Valencia,2325.0,NaN,NaN,NaN,NaN,NaN,NaN,47.96,NaN
639,Unincorporated - Valinda,18507.0,NaN,NaN,NaN,NaN,NaN,NaN,35.58,NaN
641,Unincorporated - View Park/Windsor Hills,10084.0,NaN,NaN,NaN,NaN,NaN,NaN,42.92,NaN
643,Unincorporated - Walnut Park,12469.0,NaN,NaN,NaN,NaN,NaN,NaN,32.53,NaN
645,Unincorporated - West Antelope Valley,1348.0,NaN,NaN,NaN,NaN,NaN,NaN,17.28,NaN
647,Unincorporated - West Carson,18482.0,NaN,NaN,NaN,NaN,NaN,NaN,44.59,NaN
649,Unincorporated - West LA,879.0,NaN,NaN,NaN,NaN,NaN,NaN,16.15,NaN


In [30]:
larger_places = df_time_pivot[df_time_pivot["population"] > 5000]

### Which places with populations of more than 5,000 saw the largest pct increase from week 8 to week 14?

In [31]:
larger_places.sort_values("change_week8_to_14", ascending=False).head(10)

,area,population,week_8,week_9,week_10,week_11,week_12,week_13,week_14,change_week8_to_14
398,Los Angeles - University Park,23588.0,19.38,45.22,53.54,74.34,102.60,140.28,80.16,314.0
516,Unincorporated - Florence-Firestone,45509.0,20.28,47.32,55.54,76.09,103.51,140.07,80.04,295.0
546,Unincorporated - Lennox,16313.0,21.87,51.03,60.57,84.42,111.57,147.77,84.44,286.0
386,Los Angeles - Thai Town,8596.0,21.63,50.47,59.67,82.67,109.04,144.20,82.40,281.0
174,Los Angeles - Alsace,9200.0,25.47,59.43,69.61,95.06,124.97,164.85,94.20,270.0
40,City of Cudahy,17477.0,21.00,49.00,58.56,82.46,105.14,135.38,77.36,268.0
212,Los Angeles - Chinatown,7292.0,32.16,75.04,89.26,124.81,159.28,205.24,117.28,265.0
248,Los Angeles - Florence-Firestone,32670.0,19.77,46.13,54.19,74.34,95.67,124.11,70.92,259.0
414,Los Angeles - Vernon Central,35744.0,19.56,45.64,53.30,72.45,93.87,122.43,69.96,258.0
540,Unincorporated - Lake Los Angeles,9635.0,17.58,41.02,47.46,63.56,83.48,110.04,62.88,258.0


### Which places with populations of more than 5,000 saw the smallest pct increase from week 8 to week 14?

In [32]:
larger_places.sort_values("change_week8_to_14", ascending=False).tail(10)

,area,population,week_8,week_9,week_10,week_11,week_12,week_13,week_14,change_week8_to_14
620,Unincorporated - South San Gabriel,7548.0,NaN,NaN,NaN,NaN,NaN,NaN,46.36,NaN
622,Unincorporated - South Whittier,47333.0,NaN,NaN,NaN,NaN,NaN,NaN,32.48,NaN
626,Unincorporated - Stevenson Ranch,17460.0,NaN,NaN,NaN,NaN,NaN,NaN,40.84,NaN
639,Unincorporated - Valinda,18507.0,NaN,NaN,NaN,NaN,NaN,NaN,35.58,NaN
641,Unincorporated - View Park/Windsor Hills,10084.0,NaN,NaN,NaN,NaN,NaN,NaN,42.92,NaN
643,Unincorporated - Walnut Park,12469.0,NaN,NaN,NaN,NaN,NaN,NaN,32.53,NaN
647,Unincorporated - West Carson,18482.0,NaN,NaN,NaN,NaN,NaN,NaN,44.59,NaN
651,Unincorporated - West Puente Valley,7882.0,NaN,NaN,NaN,NaN,NaN,NaN,36.35,NaN
655,Unincorporated - West Whittier/Los Nietos,21564.0,NaN,NaN,NaN,NaN,NaN,NaN,37.27,NaN
666,Unincorporated - Willowbrook,26433.0,NaN,NaN,NaN,NaN,NaN,NaN,24.86,NaN


---

### Export to csv

In [33]:
import datetime as dt

today = dt.datetime.today().strftime("%m-%d-%Y")

In [34]:
# df.to_csv(
#     "output/lacounty_vaccinations_by_city_community_" + today + ".csv", index=False
# )

In [35]:
concatenated_df.to_csv(
    "output/lacounty_vaccinations_by_city_community_timeseries_" + today + ".csv",
    index=False,
)

In [36]:
df_time_pivot.to_csv(
    "output/lacounty_vaccinations_by_city_community_weeks_" + today + ".csv",
    index=False,
)